# 12.0 Recursion

## 12.1 Towers of Hanoi

### Problem Statement
Imagine 3 rods with $n$ discs of increasing diameter stacked on the first rod.  Write a function that moves all $n$ discs from the first rod to a goal rod according to the following rules:
* A disc moves from the top of one rod to the top of another rod.
* A smaller disc can only be placed on top of a wider disc.
* Only 1 disc can be moved at a time.

The solution should be returned as a list of tuples of $(d, r_s, r_d)$ where:
* $d$ is the diameter of the disc to move
* $r_s$ and $r_d$ are the source and destination rods respectively

In [1]:
from collections import deque, namedtuple
import unittest


def make_rods(n):
    """Return rods initialized with n discs."""
    rods = [deque(), deque(), deque()]
    for d in range(n,0,-1):
        rods[0].appendleft(d)
    return rods


def rod_index(rods, d):
    """Return rod index containing d as topmost disc or None."""
    for ind, rod in enumerate(rods):
        if len(rod) > 0 and rod[0] == d:
            return ind
    return None


def move(rods, rs, rd):
    """Move topmost disc from rs to rd."""
    rods[rd].appendleft(rods[rs].popleft())


def towers_of_hanoi(rods, n, right=True):
    """Return the solution tuple for the towers of hanoi with n discs."""

    moves = []
    if n < 1:
        return moves

    # Move all smaller discs (<n) to some other rod.
    moves.extend(towers_of_hanoi(rods, n-1, not(right)))

    # Move the nth disc to the goal rod.
    # Goal rod rotates left or rotate at each level of recursion.
    rs = rod_index(rods, n)
    rd = (rs+1)%len(rods) if right else (rs-1)%len(rods)
    move(rods, rs, rd)
    moves.append((n, rs, rd))

    # Move all smaller discs back to the goal rod.
    moves.extend(towers_of_hanoi(rods, n-1, not(right)))

    return moves


class TowersOfHanoiTest(unittest.TestCase):

    def test_towers_of_hanoi(self):
        case = namedtuple('case', ['n','expected'])
        cases = [
            # Edge cases.
            case(0, []),
            case(1, [(1,0,1)]),
            case(2, [(1,0,2),(2,0,1),(1,2,1)]),
            # Standard case: 3 discs.
            case(3, [(1,0,1),(2,0,2),
                     (1,1,2),(3,0,1),
                     (1,2,0),(2,2,1),
                     (1,0,1)]),
            # 4 discs = f(3 discs) + 4 + f(3 discs).
            case(4, [(1,0,2),(2,0,1),
                     (1,2,1),(3,0,2),
                     (1,1,0),(2,1,2),
                     (1,0,2),
                     (4,0,1),(1,2,1),
                     (2,2,0),(1,1,0),
                     (3,2,1),(1,0,2),
                     (2,0,1),(1,2,1)]),
        ]
        for c in cases:
            rods = make_rods(c.n)
            rcv = towers_of_hanoi(rods, c.n)
            self.assertEqual(rcv, c.expected)


unittest.main(TowersOfHanoiTest(), argv=[''], verbosity=2, exit=False)

test_towers_of_hanoi (__main__.TowersOfHanoiTest) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.005s

OK


## 12.2 Implement regular expression

### Problem Statement
Implement a regular expression matcher with the following rules:
1. `.` matches any single character
1. `*` matches 0 or more copies of the previous character

#### Examples

| Pattern | Match | No Match |
| ------- | ----- | -------- |
| ra. | ray | raymond |
| fo\*d | foood | fooods |
| .\*at | chat | chats |

In [2]:
from collections import namedtuple
import unittest


def match(pattern, string):
    """Return True when string matches pattern."""
    #print(pattern, string)
    
    # Matching is over when either of the pattern or string ends.
    if len(pattern) == 0 or len(string) == 0:
        return True if pattern == string or pattern == '*' else False

    # Wildcard match.
    if len(pattern) > 2 and pattern[1] == '*':
        ind = 0
        if pattern[:2] == '.*':
            # Advance string to first character after wildcard.
            while ind < len(string) and string[ind] != pattern[2]:
                ind += 1
        else:
            # Advance string to the first non-repeating character.
            while ind < len(string) and string[ind] == pattern[0]:
                ind += 1
        return match(pattern[2:], string[ind:])
    # Exact match or single character wildcard.
    elif pattern[0] == string[0] or pattern[0] == '.':
        return match(pattern[1:], string[1:])

    return False


class MatchTest(unittest.TestCase):
    
    def test_match(self):
        case = namedtuple('case', ['pattern','string','expected'])
        cases = [
            # Simple exact string matches.
            case('rat', 'rat', True),
            case('rat', 'rad', False),
            case('rat', 'bat', False),
            # Match contains '.'.
            case('ra.', 'ray', True),
            case('ra.', 'raymond', False),
            case('r.t', 'rot', True),
            case('r.t', 'rat', True),
            case('.at', 'hat', True),
            case('.at', 'hats', False),
            # Match contains '*'.
            case('fo*d', 'fod', True),  # 1 occurrence.
            case('fo*d', 'foood', True),  # 3 occurrences.
            case('fo*d', 'fd', True),  # 0 occurrences.
            case('fo*d', 'fooods', False),
            case('tail*', 'tail', True),
            # Match contains mix of '.' and '*'.
            case('.*at', 'chat', True),
            case('.*at', 'helloat', True),
            case('.*at', 'hellot', False),
            case('.*at', 'chats', False),
        ]
        for c in cases:
            rcv = match(c.pattern, c.string)
            self.assertEqual(rcv, c.expected)


unittest.main(MatchTest(), argv=[''], verbosity=2, exit=False)

test_match (__main__.MatchTest) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.010s

OK


## 12.3 Find minimum and maximum efficiently

### Problem Statement
Given an array of $n$ numbers, find the minimum and maximum using less than $2(n-2)$ comparisons.

In [3]:
from collections import namedtuple
import unittest


def minmax(elems):
    """Return a tuple of the minimum and maximum elements."""
    
    # Use divide and conquer to find minimum and maximum
    # of left and right subarrays.  During the merge, only
    # the extreme values are compared, thus avoiding an
    # additional comparison per recursive step resulting in
    # (3n/2)-2 comparisons instead of 2n-2.
    if len(elems) == 1:
        return (elems[0], elems[0])
    elif len(elems) == 2:
        return ((elems[0], elems[1]) if elems[0] < elems[1] else
                (elems[1], elems[0]))
    
    mid = len(elems)//2
    min1, max1 = minmax(elems[:mid])
    min2, max2 = minmax(elems[mid:])
    return (min(min1, min2), max(max1, max2))


class MinmaxTest(unittest.TestCase):
    
    def test_minmax(self):
        case = namedtuple('case', ['input', 'expected'])
        cases = [
            case([1],(1,1)),
            # Permutations of [1,2].
            case([1,2],(1,2)),
            case([2,1],(1,2)),
            # Permutations of [1,2,3].
            case([1,2,3],(1,3)),
            case([1,3,2],(1,3)),
            case([2,1,3],(1,3)),
            case([2,3,1],(1,3)),
            case([3,1,2],(1,3)),
            case([3,2,1],(1,3)),
            # Random.
            case([18,44,29,50,49,43,2,67,3,81], (2,81)),
            case([3,2,44,18,29,50,43,49,81,67], (2,81)),
            case([29,18,2,43,50,81,3,67,49,44], (2,81)),
        ]
        for c in cases:
            rcv = minmax(c.input)
            self.assertEqual(rcv, c.expected)


unittest.main(MinmaxTest(), argv=[''], verbosity=2, exit=False)

test_minmax (__main__.MinmaxTest) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.013s

OK
